In [2]:
import requests, json
from collections import OrderedDict
from operator import getitem
import time
import pandas as pd
from web3 import Web3
import warnings
import struct
import binascii

In [21]:
!pip3 install --upgrade eth-abi

  Attempting uninstall: eth-abi
    Found existing installation: eth-abi 4.0.0
    Uninstalling eth-abi-4.0.0:
      Successfully uninstalled eth-abi-4.0.0

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
wayvern_contract = Web3.to_checksum_address('0x00000000006c3852cbef3e08e8df289169ede581')
orders_matched_event = '0x9d9af8e38d66c62e2c12f0225249fd9d721c54b83f48d9352c97c6cacdcb6f31'
api_key = 'UAK615004150ad5a46f828e9b9339c133df'

In [4]:
# Replace with your AMBERDATA API KEY
eth_node_url = f'https://rpc.web3api.io/api/v2?x-api-key={api_key}'

In [5]:
w3 = Web3(Web3.HTTPProvider(eth_node_url))

# Check if connected to Ethereum node
if w3.is_connected:
    print("Connected to Ethereum node")
else:
    print("Not connected to Ethereum node")
    exit(1)

Connected to Ethereum node


In [6]:
headers = {
    "accept": "application/json",
    "x-amberdata-blockchain-id": "ethereum-mainnet",
    "x-api-key": f'{api_key}'
}

In [7]:
#Fetch Contract ABI: 
abi_url = f'https://web3api.io/api/v2/contracts/{wayvern_contract}' 

In [8]:
#Fetching contract address ABI, formating and extracting ABI Only. Will use later for parsing logs. 
abi = requests.request("GET", url=abi_url, headers=headers)
abi = json.loads(abi.text)
abi = abi['payload']['abi']

In [12]:
abi

[{'inputs': [{'internalType': 'address',
    'name': 'conduitController',
    'type': 'address'}],
  'stateMutability': 'nonpayable',
  'type': 'constructor'},
 {'inputs': [], 'name': 'BadContractSignature', 'type': 'error'},
 {'inputs': [], 'name': 'BadFraction', 'type': 'error'},
 {'inputs': [{'internalType': 'address', 'name': 'token', 'type': 'address'},
   {'internalType': 'address', 'name': 'from', 'type': 'address'},
   {'internalType': 'address', 'name': 'to', 'type': 'address'},
   {'internalType': 'uint256', 'name': 'amount', 'type': 'uint256'}],
  'name': 'BadReturnValueFromERC20OnTransfer',
  'type': 'error'},
 {'inputs': [{'internalType': 'uint8', 'name': 'v', 'type': 'uint8'}],
  'name': 'BadSignatureV',
  'type': 'error'},
 {'inputs': [],
  'name': 'ConsiderationCriteriaResolverOutOfRange',
  'type': 'error'},
 {'inputs': [{'internalType': 'uint256',
    'name': 'orderIndex',
    'type': 'uint256'},
   {'internalType': 'uint256',
    'name': 'considerationIndex',
    'ty

In [25]:
#Empyt dict for storing returned data. 
collection = {}
id = 0
#For Pagination
pagination = 0
total_pages = 1
paginationIncrement = 1

In [26]:
#Create Web3 Contract Instance: 
myContract = w3.eth.contract(address=beacon_chain_contract, abi=abi)

In [18]:
#Alternative Approach is using the ABI + Web3.py to Process_Receipt - This approach is a bit slower since it request an RPC call. 
while pagination <= total_pages:
    #Fetch Contract Events Logs. 
    url = f'https://web3api.io/api/v2/addresses/{beacon_chain_contract}/logs?topic={beacon_chain_deposit_event}&page={pagination}&size=1'
    response = requests.request("GET", url, headers=headers)
    while response.status_code == 429:
        time.sleep(1)
        response = requests.request("GET", url, params=querystring)
    batch_list = json.loads(response.text)["payload"]['records']
    for item in batch_list:
        try: 
            print(item["transactionHash"])
            receipt = w3.eth.get_transaction_receipt(item["transactionHash"])
            logs = myContract.events.OrderFulfilled().process_receipt(receipt)
            id +=1
            collection[id] = {}
        

            print("AD Logs", item)
            collection[id]['offer_ad'] = item['data']
            
        except NameError:
            print(NameError)
        except:
            print("No Events at this TX")
    print('Finished page',pagination)
    pagination += paginationIncrement


0x8605571efe3144b47b6157a3d50c20d8fef99b901c466dc65b8f32e6430769ee
my logs (AttributeDict({'args': AttributeDict({'offerer': '0x4b2C444ba3a88d26825CcC310AB54AD679DC1949', 'zone': '0x0000000000000000000000000000000000000092', 'orderHash': b'\xb3j{\x9f10\xaf\xcd8\x0c`\x97\x08\xff\xc0\xcd(G5\x8a\xe1l\x90\x18\xfe\xee\xbf,\x84O\xecC', 'recipient': '0x864e3aF0D1a532BB92E6DA167F67d2bd033aF00F', 'offer': [AttributeDict({'itemType': 2, 'token': '0x3d049aDb773fADDeF681FbE565466C4F9736A009', 'identifier': 3434, 'amount': 1})], 'consideration': [AttributeDict({'itemType': 0, 'token': '0x0000000000000000000000000000000000000000', 'identifier': 0, 'amount': 83600000000000000, 'recipient': '0x4b2C444ba3a88d26825CcC310AB54AD679DC1949'}), AttributeDict({'itemType': 0, 'token': '0x0000000000000000000000000000000000000000', 'identifier': 0, 'amount': 2200000000000000, 'recipient': '0x8CA8A587710CEDa4232AC33d9dA8b7421ffc90be'}), AttributeDict({'itemType': 0, 'token': '0x00000000000000000000000000000000000

In [22]:
from eth_abi import decode

In [27]:
# Define the ABI of the event you want to decode
abi = [
    {'anonymous': False,
  'inputs': [{'indexed': False,
    'internalType': 'bytes32',
    'name': 'orderHash',
    'type': 'bytes32'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'offerer',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'zone',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'address',
    'name': 'recipient',
    'type': 'address'},
   {'components': [{'internalType': 'enum ItemType',
      'name': 'itemType',
      'type': 'uint8'},
     {'internalType': 'address', 'name': 'token', 'type': 'address'},
     {'internalType': 'uint256', 'name': 'identifier', 'type': 'uint256'},
     {'internalType': 'uint256', 'name': 'amount', 'type': 'uint256'}],
    'indexed': False,
    'internalType': 'struct SpentItem[]',
    'name': 'offer',
    'type': 'tuple[]'},
   {'components': [{'internalType': 'enum ItemType',
      'name': 'itemType',
      'type': 'uint8'},
     {'internalType': 'address', 'name': 'token', 'type': 'address'},
     {'internalType': 'uint256', 'name': 'identifier', 'type': 'uint256'},
     {'internalType': 'uint256', 'name': 'amount', 'type': 'uint256'},
     {'internalType': 'address payable',
      'name': 'recipient',
      'type': 'address'}],
    'indexed': False,
    'internalType': 'struct ReceivedItem[]',
    'name': 'consideration',
    'type': 'tuple[]'}],
  'name': 'OrderFulfilled',
  'type': 'event'}
]

# The raw log data from the Ethereum transaction
raw_log_data = "b36a7b9f3130afcd380c609708ffc0cd2847358ae16c9018feeebf2c844fec43000000000000000000000000864e3af0d1a532bb92e6da167f67d2bd033af00f00000000000000000000000000000000000000000000000000000000000000800000000000000000000000000000000000000000000000000000000000000120000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000020000000000000000000000003d049adb773faddef681fbe565466c4f9736a0090000000000000000000000000000000000000000000000000000000000000d6a00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000003000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000012901c1cf3900000000000000000000000000004b2c444ba3a88d26825ccc310ab54ad679dc19490000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000007d0e36a8180000000000000000000000000008ca8a587710ceda4232ac33d9da8b7421ffc90be0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000007d0e36a8180000000000000000000000000001bad6364b01013ff20193bf6562db1247fe342ed"

In [30]:
# Decode the log data using the ABI
decoded_data = decode(abi[0]["inputs"], bytes.fromhex(raw_log_data))

TypeError: The `types` value type must be one of list or tuple. Got <class 'str'>

In [29]:
j =['0x00003A01C01173D676B644D4E040DB9CC01173D676B62069311401000009416E64726F69644150010882848B962430486C32040C12186030140100000FAC040100000FAC040100000FAC020C002D1A2C0103FF00000000000000000000000000000000000000000000DD070050F2020001007F0400000A02']

hex_to_ascii = bytes.fromhex(j[0][2:])

print(hex_to_ascii)

b'\x00\x00:\x01\xc0\x11s\xd6v\xb6D\xd4\xe0@\xdb\x9c\xc0\x11s\xd6v\xb6 i1\x14\x01\x00\x00\tAndroidAP\x01\x08\x82\x84\x8b\x96$0Hl2\x04\x0c\x12\x18`0\x14\x01\x00\x00\x0f\xac\x04\x01\x00\x00\x0f\xac\x04\x01\x00\x00\x0f\xac\x02\x0c\x00-\x1a,\x01\x03\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xdd\x07\x00P\xf2\x02\x00\x01\x00\x7f\x04\x00\x00\n\x02'
